In [1]:
import gzip
import os
import json
import pandas as pd
from tqdm import tqdm

import plotly.express as px

import numpy as np
from sklearn import preprocessing

from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

from sklearn.metrics import accuracy_score

import shutil

from processDataset import *
from datetime import datetime

from sklearn.model_selection import train_test_split

In [2]:
DATA_DIR = '../carat-data-top1k-users-2014-to-2018-08-25/top1k-salted-data-to-share-2014-01-01-to-2018-08-25.json-rdd/'
output_dir = "output"

if False:    
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)

    os.mkdir(output_dir)
    preprocess_dataset(DATA_DIR, output_dir)

In [3]:
user_summary = get_user_summary(output_dir)

100%|██████████| 1000/1000 [01:36<00:00, 10.35it/s]

Total users : 1000


In [5]:
px.histogram(x=[len(uuid['unique_apps']) for uuid in user_summary.values()], title="Number of unique apps per user")

In [4]:
sorted_users_by_event = {k:v for k,v in sorted(user_summary.items(), key=lambda item: item[1]['event_count'], reverse=True)}

In [65]:
data = get_user_data(list(sorted_users_by_event.keys())[1], output_dir)

In [66]:
X, y = get_X_y(data, 50)

100%|██████████| 287988/287988 [00:02<00:00, 134684.44it/s]


In [7]:
numSame = 0
numDiff = 0

num = 0

prev = None
for i in tqdm(range(y.shape[0])):
    ele = list(y.iloc[i])
    if prev == None:
        prev = list(ele)
    else:
        if prev == list(ele):
            numSame += 1
        else:
            numDiff += 1
    num += 1

print(numSame)
print(numDiff)
print(numSame / num)
print(num)

100%|██████████| 290447/290447 [00:14<00:00, 20579.99it/s]

78068
212378
0.26878569928420676
290447


In [67]:
prev = None
to_drop = []
for i in tqdm(range(y.shape[0])):
    ele = list(y.iloc[i])
    if prev == None:
        prev = list(ele)
    else:
        if prev == list(ele):
            to_drop.append(i)

dataframe_drop = []
x_df_drop = []
for i in to_drop:
    dataframe_drop.append(y.index[i])
    x_df_drop.append(X.index[i-1])

nodup_y = y.drop(dataframe_drop).reset_index().drop(['index'], axis=1)
nodup_x = X.drop(x_df_drop).reset_index().drop(['index'], axis=1)

100%|██████████| 287987/287987 [00:14<00:00, 20149.20it/s]


In [68]:
y_new = nodup_y - nodup_x.iloc[:,3:]

In [69]:
y_new_nozero = y_new.loc[~(y_new==0).all(axis=1)]
X_nozero = nodup_x.loc[~(y_new==0).all(axis=1)]

In [70]:
y_new_nozero.sum(axis=0)

com.google.android.youtube                           2
com.sec.android.app.launcher                         2
com.android.chrome                                   2
android                                              1
com.nam.fbwrapper                                    0
com.android.systemui                                -1
com.android.vending                                  0
com.google.android.googlequicksearchbox              2
com.google.android.gm                                0
org.wikipedia                                        0
com.android.settings                                 0
com.whatsapp                                         1
com.google.android.apps.maps                         0
com.google.android.gms                               1
com.google.android.apps.docs                         0
com.google.android.apps.genie.geniewidget            0
com.ea.game.simpsons4_row                            6
com.sec.android.gallery3d                           -1
com.samsun

In [37]:
y_new_nozero = y_new_nozero.drop(['com.android.phone', 'com.android.nfc', 'com.cyngn.keyguard', 'com.android.incallui', 'com.android.systemui', 'com.cyngn.maxxaudio'], axis=1)

In [41]:
X_nozero = X_nozero.drop(['com.android.phone', 'com.android.nfc', 'com.cyngn.keyguard', 'com.android.incallui', 'com.android.systemui', 'com.cyngn.maxxaudio'], axis=1)

In [71]:
y_new_nozero[y_new_nozero < 0].sum(axis=0)

com.google.android.youtube                         -205.0
com.sec.android.app.launcher                       -260.0
com.android.chrome                                 -518.0
android                                            -115.0
com.nam.fbwrapper                                  -146.0
com.android.systemui                               -116.0
com.android.vending                                -260.0
com.google.android.googlequicksearchbox            -106.0
com.google.android.gm                               -99.0
org.wikipedia                                      -140.0
com.android.settings                               -117.0
com.whatsapp                                        -68.0
com.google.android.apps.maps                        -63.0
com.google.android.gms                              -93.0
com.google.android.apps.docs                        -36.0
com.google.android.apps.genie.geniewidget           -33.0
com.ea.game.simpsons4_row                          -236.0
com.sec.androi

In [52]:
X.shape
nodup_x.shape

(212478, 103)

In [15]:
y.shape

(290447, 100)

In [30]:
X = nodup_x
y = nodup_y

In [87]:
X_nozero.shape

(4114, 53)

Neural Network

In [72]:
from sklearn.neural_network import MLPRegressor


In [108]:
X_train, X_test, y_train, y_test = train_test_split(X_nozero.iloc[:1000], y_new_nozero.iloc[:1000], test_size=.1, shuffle=False, random_state=42)


mlp = MLPRegressor(alpha=.0005)
mlp.fit(X_train.values, y_train.values)


MLPRegressor(alpha=0.0005)

In [109]:
preds = mlp.predict(X_test.values).round()


In [85]:
print(preds)

[[ 0.  0.  0. ... -0.  0. -0.]
 [-0.  0.  0. ...  0.  0. -0.]
 [-0.  0.  0. ...  0.  0. -0.]
 ...
 [ 0.  0.  0. ... -0. -0. -0.]
 [ 0. -0.  1. ... -0.  0. -0.]
 [ 0.  0.  0. ... -0. -0. -0.]]


# Ridge Regression

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X_nozero, y_new_nozero, test_size=.1, shuffle=True, random_state=42)

print(f"Total train size : {len(X_train)}")
print(f"Total test size : {len(X_test)}")

Total train size : 16943
Total test size : 1883


In [61]:
y_test

,system,com.cyanogenmod.trebuchet,com.android.chrome,com.google.android.gms.persistent,com.google.android.gm,com.android.deskclock,com.urbandroid.lux:background,com.google.android.apps.maps,com.google.android.talk,android.process.acore,...,com.google.android.apps.plus,com.android.dialer,com.google.android.gms,com.urbandroid.lux,com.ubercab,no.nrk.yr,com.cyngn.fota,com.google.android.music,com.google.earth,com.cyanogenmod.lockclock
13074,0,0,0,0,-1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
21299,0,0,-1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1470,0,-1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17813,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21992,0,0,0,0,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
209651,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
42340,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [111]:
regr = MultiOutputRegressor(Ridge(random_state=123)).fit(X_train.values, y_train.values)
# regr = MultiOutputRegressor(Ridge(random_state=123)).fit(X_train.values, y_train.values)

# preds = pred_to_df(regr.predict(X_test.values).round(), y_test.columns)
preds = regr.predict(X_test.values).round()
print(preds.shape)
print(y_test.shape)

# print(f"Accuracy is : {accuracy_score(preds, y_test.values)*100}%")

(100, 50)
(100, 50)


In [112]:
preds = regr.predict(X_test.values).round()

# print(f"Accuracy is : {accuracy_score(preds, y_test.values)*100}%")

In [81]:
preds

array([[ 0.,  0.,  0., ..., -0.,  0., -0.],
       [-0.,  0.,  0., ...,  0.,  0., -0.],
       [-0.,  0.,  0., ...,  0.,  0., -0.],
       ...,
       [ 0.,  0.,  0., ..., -0., -0., -0.],
       [ 0., -0.,  1., ..., -0.,  0., -0.],
       [ 0.,  0.,  0., ..., -0., -0., -0.]])

In [113]:
right = 0
wrong = 0
vals = y_test.values
for i in range(len(preds)):
    if list(preds[i]) == list(vals[i]):
        right += 1
    else:
        wrong += 1

print(right)
print(wrong)

print (right / float(right + wrong))

21
79
0.21


In [80]:
right = 0
wrong = 0
vals = y_test.values
for i in range(len(preds)):
    for j in range(len(preds[i])):
        if vals[i][j] != 0:
            if preds[i][j] == vals[i][j]:
                right += 1
            else:
                wrong += 1

print(right)
print(wrong)

print (right / float(right + wrong))

211
439
0.32461538461538464


linear reg:
3930
4727
608

neural reg (80% test):
5587
37148
1601

neural reg (50% test):
56888
36132
1058

In [63]:
wrong_prefetch = 0
wrong_evict = 0
num_right = 0
y_val = y.values
for i in tqdm(range(1, len(y))):
    for j in range(preds.shape[1]):
        if preds[i][j] == 1 and y_val[i][j] == 0:
            wrong_prefetch += 1
        elif preds[i][j] == 0 and y_val[i][j] == 1:
            wrong_evict += 1
        else:
            if y_val[i-1][j] != y_val[i][j]:
                num_right += 1
                
print(wrong_prefetch)
print(wrong_evict)
print(num_right)

  1%|          | 1882/290446 [00:00<00:19, 14945.57it/s]


IndexError: index 1883 is out of bounds for axis 0 with size 1883

In [53]:
def get_new_apps(prev, current):
    return list(set(prev) - set(current))

def get_closing_apps(prev, current):
    return list(set(current) - set(prev))

In [30]:
new_apps = []
closing_apps = []
for (_, X_sample), (_, y_sample) in zip(X.iterrows(), y.iterrows()):
    prev = list(X_sample[X_sample == 1][1:].keys())
    current = list(y_sample[y_sample == 1][1:].keys())
    new_apps.append(get_new_apps(prev, current))
    closing_apps.append(get_closing_apps(prev, current))

In [33]:
new_apps[:10]

[['com.android.incallui', 'edu.berkeley.cs.amplab.carat.android'],
 ['com.android.incallui'],
 ['com.cyanogenmod.trebuchet', 'com.android.incallui'],
 ['com.android.incallui'],
 ['com.android.incallui'],
 ['com.android.incallui'],
 ['com.android.incallui', 'android.process.acore', 'com.google.android.talk'],
 ['com.android.incallui'],
 ['com.android.incallui'],
 ['com.android.incallui']]

In [34]:
closing_apps[:10]

[['com.cyanogenmod.trebuchet'],
 [],
 ['android.process.acore', 'com.google.android.talk'],
 [],
 [],
 [],
 ['com.google.android.gm'],
 [],
 [],
 []]